![Podmanlogo](Pictures/podman-logo.png)

# Podman ecosystem

We have seen what an awsome piece of software Podman is, but that is not all. Podman counts with some other tools in its ecosystem that makes everything run smoothly.
Lets review all of them.

## Managing multiple containers: podman compose and kubernetes

Running a single container with podman is just fine, we have demonstrated how simple it is througout this workshop. But, what happens if I want to run multiple containers simultaneously? Do you need to execute them one by one?

You can use the tool podman compose to execute multiple containers in a single command. Podman compose is a tool that helps you defining the list of containers and all of their configuration in a text file written in yaml format. Then you can choose to execute, stop, remove, update all of the containers defined in the file at once. Podman compose is a very powerful tool but it's limited to execute on a single system.

Lets learn how podman-compose can help you deploying multiple containers at once. First we are going to build two containers called "container-A" and "container-B", for this we will use the same Containerfile that we used in the section 2 of this workshop. But this time the web service will answer "I'm SERVICE A running in container-A" when doing a curl to container-A and "I'm SERVICE B running in container-B" when running curl against container-B.

In [ ]:
%login student491@16.31.86.187

In [ ]:
echo "I'm SERVICE A running in container-A" > indexA.html
echo "I'm SERVICE B running in container-B" > indexB.html
echo -e "FROM docker.io/redhat/ubi9\nRUN dnf -y update && dnf -y install httpd  && dnf clean all\nCOPY indexA.html /var/www/html/index.html\nEXPOSE 80\nENTRYPOINT /usr/sbin/httpd -DFOREGROUND" > ContainerfileA
echo -e "FROM docker.io/redhat/ubi9\nRUN dnf -y update && dnf -y install httpd  && dnf clean all\nCOPY indexB.html /var/www/html/index.html\nEXPOSE 80\nENTRYPOINT /usr/sbin/httpd -DFOREGROUND" > ContainerfileB
podman build --tag container-a:v1 -f ./ContainerfileA
podman build --tag container-b:v1 -f ./ContainerfileB

Check both your images have been created:

In [ ]:
podman images

Now, we can create a compose file and create both containers at once. First lets create the file and then we will see what's in it.

In [ ]:
echo -e "services:\n  service-A:\n      image: localhost/container-a:v1\n      container_name: container-A\n      restart: always\n      ports:\n        - "8080:80"\n\n  service-B:\n      image:     localhost/container-b:v1\n      container_name: container-B\n      restart: always\n      ports:\n        - "8081:80"\n      depends_on:\n        - service-A\n" > compose.yml
cat compose.yml

As you can see the file that we just created is using the Yaml format, meaning that indentation defines what objects are at the same level or what lines are options inside certain object.

In our example, as you can see in the output of previous command, you can see that first we have a line that states "services" and then we have two lines "service-A" and "service-B", as the last two lines are indented inside the "services" section podman-compose will know that they are services objects. Then we have a few options within each service that define how they will need to be deployed. Both "service-A" and "service-B" are arbitrary names.

Within each service we define the container image to be used with the "image" option, the container name with "container_name" option, the ports to be exposed with the "ports" option and the restart policy with the "restart" option.

In the service-B service we defined the option "depends_on" stating that service-B will only be started once service-A has started. In our application this doesn't matter, but many applications will have services that depend on other services.

Now we can just run the command to execute both containers.

In [ ]:
podman-compose up -d

Now check both of your containers are running.

In [ ]:
podman ps

You see how easy it is to create multiple containers with podman-compose. Now lets stop both of them.

In [ ]:
podman-compose down

Check they have been stopped.

In [ ]:
podman ps

In this example we have used using two web server containers, but think about how useful this tool would be for a classic three tier app. For example one having a frontend, a backend and a database. You could deploy all of the pieces in the right order in a super easy and repeatable way with podman-compose.

Podman-compose is a very interesting tool, but it's limited to deploying containers to a single system. If you're looking at deploying your containers in multiple systems simultaneously in order to get scalability and high availability then you will most probably end up using kubernetes as it has become the standard platform for running containerized workloads in the enterprise space.
Kubernetes, same as podman composer, uses yaml files to define the desired state of your containerized workloads and, once you pass the file to the control plane nodes of kubernetes, it will make sure your containers run according to your specifications. Furthermore they will be spread accross all the nodes that are part of your kubernetes cluster.

![k8slogo](Pictures/kubernetes-logo.png)

Now, transitioning from single system container engines like Podman or Docker to kubernetes may not always be easy. That is why Podman can generate a kubernetes yaml file from the container that you are running righ now with Podman. First you just need to execute a new container:

In [ ]:
podman run -d --rm --name=kubepodman docker.io/redhat/ubi9 sleep 999

Now your container is running:

In [ ]:
podman ps

With this, you can execute the "podman generate kube" command to export a yaml file, which you can use as definition for your kubernetes environment.

In [ ]:
podman generate kube kubepodman > kubepodman.yml
cat kubepodman.yml

The output of the previous command is the content of the yaml file that you need to use for running your container in a kubernetes workload. At the beginning of the file you can even see a few comments in which you find the command you need to run in order to start executing this container in your kubernetes cluster: "kubectl create -f kubepodman.yml".

As you can see, podman has multiple tools that allow you to transition to managing multiple containers at once and simplify the transition to those tools.

# Cleanup

In [ ]:
podman stop --all
podman rm --all
podman rmi docker.io/redhat/ubi9 localhost/container-a:v1 localhost/container-a:v1
rm kubepodman.yml indexA.html indexB.html ContainerfileA ContainerfileB compose.yml

In [ ]:
%logout